# Slowed and Reverb Music Video Maker

References to project:
- [Youtube video downloader on SO](https://stackoverflow.com/questions/65802599/how-to-download-youtube-video-in-the-highest-quality-available)
- [File Chooser widget](https://pypi.org/project/ipyfilechooser/)

References for download progress bar from: [1](https://stackoverflow.com/questions/56197879/how-to-use-progress-bar-in-pytube) and [2](https://stackoverflow.com/questions/56197879/how-to-use-progress-bar-in-pytube).

pip installs:
- pytube
- ipyfilechooser
- moviepy

Import relevant libraries for this project.

In [29]:
#YouTube library
import pytube
from pytube.cli import on_progress
import sys
from ipyfilechooser import FileChooser
import ipywidgets as widgets

### Specify the PATH for the videos to be saved here:

In [2]:
PATH = input("> ")

> C:\Users\asher\Downloads


### Insert the URL of the song you want to convert. 

In [4]:
#video downloader
def download():
    video_url = url
    try:
        youtube = pytube.YouTube(video_url, on_progress_callback = progress_func)
        video = youtube.streams.last()
        video.download(PATH)
        print("Download Complete!   ")
    except Exception as e:
        print(e)
        print("Video could not be downloaded.")

#progress bar function        
def progress_func(stream, chunk, bytes_remaining):
    current = ((stream.filesize - bytes_remaining)/stream.filesize)
    percent = ('{0:.1f}').format(current * 100)
    progress = int(50 * current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar = status, percent = percent))
    sys.stdout.flush()
    
url = input("> ")
download()

> https://www.youtube.com/watch?v=omT5rKmU5_k
Download Complete!████████████████████████████████████| 100.0%


### Select a GIF to loop for the entire duration of the video.

In [36]:
fc = FileChooser()
# Can put PATH into FileChooser argument if GIFs are saved in the same directory as the .WEBM files
display(fc)

FileChooser(path='C:\Users\asher\OneDrive\personal projects\snr maker', filename='', title='', show_hidden=Fal…

In [42]:
print('You have selected: \n')
print(fc.selected_filename)
selected_gif_path = fc.selected

file = open(selected_gif_path, "rb")
image = file.read()
widgets.Image(
    value=image,
    format='gif',
    width=300,
    height=400,
)

You have selected: 

aesthetic-anime-anime.gif


Image(value=b'GIF89a\xdc\x00Y\x00\xf7\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\…

In [ ]:
import moviepy.editor as mp

video = mp.VideoFileClip("video.mp4")

logo = (mp.ImageClip("logo.png")
          .set_duration(video.duration)
          .resize(height=50) # if you need to resize...
          .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
          .set_pos(("center","center")))

final = mp.CompositeVideoClip([video, logo])
final.write_videofile("test.mp4")